# Fine-tuning for parenthesis balancing

## Setup

In [1]:
RESULTS_DATA_DIR = "../results"
FINE_TUNE_PREFIX = "fine_tune_paren_bal"

In [11]:
import json
import os
from collections import OrderedDict

import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as colors

from rich.table import Table
from rich.console import Console

In [3]:
import plotly.io as pio
pio.renderers.default = "colab+vscode"

console = Console()

## Results

In [4]:
data = []
for filename in os.listdir(RESULTS_DATA_DIR):
    filepath = os.path.join(RESULTS_DATA_DIR, filename)
    if os.path.isfile(filepath) and filename.startswith(FINE_TUNE_PREFIX):
        with open(filepath) as fp:
            data.append(json.load(fp))

In [18]:
learning_rates = []
val_accuracy_per_lr = {}
train_accuracies_per_lr = {}
for datum in data:
    lr = datum["parameters"]["learning_rate"]
    learning_rates.append(lr)
    val_accuracy_per_lr[lr] = datum["val_accuracy"]
    train_accuracies_per_lr[lr] = datum["train_accuracies"]
learning_rates = sorted(learning_rates, reverse=True)
val_accuracy_per_lr = OrderedDict(
    [(lr, val_accuracy_per_lr[lr]) for lr in learning_rates]
)
train_accuracies_per_lr = OrderedDict(
    [(lr, train_accuracies_per_lr[lr]) for lr in learning_rates]
)

In [17]:
table = Table(
    title="Validation accuracies by learning rate", show_header=True, header_style="bold"
)
table.add_column("Learning rate", justify="right")
table.add_column("Accuracy", justify="right")
for learning_rate, accuracy in val_accuracy_per_lr.items():
    table.add_row(f"{learning_rate:.1e}", f"{accuracy:.2%}")

console.print(table)

  Validation accuracies by  
       learning rate        
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Learning rate ┃ Accuracy ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│       1.0e-01 │   55.74% │
│       1.0e-02 │   81.67% │
│       1.0e-03 │   77.76% │
│       1.0e-04 │   75.77% │
└───────────────┴──────────┘

In [24]:
px.line(
    x=learning_rates,
    y=val_accuracy_per_lr.values(),
    log_x=True,
    labels={"y": "Validation Accuracy", "x": "Learning Rate"},
    title="Validation Accuracy vs Learning Rate",
)